In [7]:
import json
import pandas as pd 
import numpy as np 
import re
import yaml
import ast
import shutil
import gzip

In [10]:
if not os.path.exists("../Datasets/reviews.json"):
    print("Extracting reviews data:...")
    with gzip.open('reviews.clean.json.gz', 'rb') as f_in:
        with open('../Datasets/reviews.json', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

if not os.path.exists("../Datasets/places.json"):
    print("Extracting places data:...")
    with gzip.open('places.clean.json.gz', 'rb') as f_in:
        with open('../Datasets/places.json', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

if not os.path.exists("../Datasets/users.json"):
    print("Extracting users data:...")
    with gzip.open('users.clean.json.gz', 'rb') as f_in:
        with open('../Datasets/users.json', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

Extracting reviews data:...
Extracting places data:...
Extracting users data:...


## Places Dataset

In [37]:
data_places = []
with gzip.open('places.clean.json.gz') as f:
    for l in f:
        data_places.append(eval(l))

In [39]:
places = pd.DataFrame(data_places)

In [40]:
places.head(10)

,name,price,address,hours,phone,closed,gPlusPlaceId,gps
0,Diamond Valley Lake Marina,None,"[2615 Angler Ave, Hemet, CA 92545]","[[Monday, [['6:30 am--4:15 pm']]], [Tuesday, [...",(951) 926-7201,False,104699454385822125632,"[33.703804, -117.003209]"
1,Blue Ribbon Cleaners,None,"[Parole, Annapolis, MD]",None,(410) 266-6123,False,103054478949000078829,"[38.979759, -76.547538]"
2,Portofino,None,"[ул. Тутаева, 1, Nazran, Ingushetia, Russia, 3...","[[Monday, [['9:30 am--9:00 pm']]], [Tuesday, [...",8 (963) 173-38-38,False,109810290098030327104,"[43.22776, 44.762726]"
3,T C's Referee Sports Bar,$$,"[5322 W 26th St, Sioux Falls, SD 57106]","[[Monday, [['11:00 am--10:00 pm']]], [Tuesday,...",(605) 361-2208,False,100327153115986850675,"[43.529494, -96.792244]"
4,Carrefour - Palembang Square,None,"[Jl. Angkatan 45, Kompleks Palembang Square, S...","[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(0711) 359918,False,103368487323937936043,"[-2.976256, 104.742662]"
5,Pizzeria Montana,None,"[Stuttgarter Straße 44, 75378 Bad Liebenzell, ...",None,07052 933486,False,103903929544828620241,"[48.767817, 8.75612]"
6,Single Tree Inn,None,"[2033 W Oklahoma Ave, Ulysses, KS 67880]",None,(620) 356-1500,False,103848836623683146185,"[37.576427, -101.386182]"
7,Sekolah Menengah Kebangsaan Tun Abang Haji Openg,None,"[Jalan Demak, Petra Jaya, Kuching, Sarawak, Ma...",None,None,False,114537653478141145113,"[1.572255, 110.362958]"
8,天山湯治郷,None,"[208 Yumotochaya, Hakone, Ashigarashimo Distri...",None,0460-86-4126,False,100062881646354125752,"[35.224978, 139.088382]"
9,Old Chicago,$$,"[17960 NW Evergreen Pkwy, Beaverton, OR 97006]","[[Monday, [['11:00 am--1:00 am']]], [Tuesday, ...",(503) 533-4650,False,118222137795476771294,"[45.535176, -122.862242]"


In [41]:
places.shape

(3114353, 8)

In [44]:
places['address'] = places['address'].apply(lambda x: " ".join(x))

In [57]:
places['City'] = places['address'].apply(lambda x: "Toronto" if x.find("Toronto") != -1 else 0)

In [58]:
toronto = places[places['City'] == 'Toronto']

In [59]:
toronto.shape

(8830, 9)

In [60]:
toronto['address']

325        170 Eglinton Ave E Toronto, ON M4P 1A6 Canada
626           900 Dufferin St Toronto, ON M6H 4A9 Canada
758             155 Bonis Ave Toronto, ON M1T 3W6 Canada
800               1075 Bay St Toronto, ON M5S 2B1 Canada
830           1300 Queen St E Toronto, ON M4L 1C4 Canada
                               ...                      
3112192      2448 Kingston Rd Toronto, ON M1N 1V3 Canada
3112655           1976 Queen St E Toronto, ON M4L Canada
3113179         5600 Yonge St Toronto, ON M2N 7L3 Canada
3113677    3400 Steeles Ave W Toronto, ON L4K 1A2 Canada
3113793         5255 Yonge St Toronto, ON M2N 6P4 Canada
Name: address, Length: 8830, dtype: object

**To do:** since Toronto is at the same location address field, we can have a better function instead of checking substrings

8830 places in Toronto

## Users

In [61]:
data_users = []
with gzip.open('users.clean.json.gz') as f:
    for l in f:
        data_users.append(eval(l))

In [62]:
users = pd.DataFrame(data_users)

## Reviews

In [63]:
data_reviews = []
with gzip.open('reviews.clean.json.gz') as f:
    for l in f:
        data_reviews.append(eval(l))

In [64]:
reviews = pd.DataFrame(data_reviews)

In [67]:
#data to work on
data = pd.merge(reviews,toronto,on = 'gPlusPlaceId',how = 'inner')
data.shape

(41061, 16)

In [81]:
data.head(20)

,rating,reviewerName,reviewText,categories,gPlusPlaceId,unixReviewTime,reviewTime,gPlusUserId,name,price,address,hours,phone,closed,gps,City
0,1.0,Vineet Udeshie,Over rated. Nothing really great about the foo...,[Mexican Restaurant],106946454276117525029,1.311991e+09,"Jul 29, 2011",100003208349643701055,Milagro,$,"5 Mercer St Toronto, ON M5V 1H2 Canada","[[Monday, [['11:30 am--10:00 pm']]], [Tuesday,...",(416) 850-2855,False,"[43.645848, -79.389668]",Toronto
1,4.0,Cathy Miller,I'm not a big fan of Mexican foods but after a...,[Mexican Restaurant],106946454276117525029,1.357108e+09,"Jan 1, 2013",100496020815170943066,Milagro,$,"5 Mercer St Toronto, ON M5V 1H2 Canada","[[Monday, [['11:30 am--10:00 pm']]], [Tuesday,...",(416) 850-2855,False,"[43.645848, -79.389668]",Toronto
2,4.0,Deepak Anand,More authentic Mexican than tex mex. Mole was...,[Mexican Restaurant],106946454276117525029,1.387464e+09,"Dec 19, 2013",102126042240566466117,Milagro,$,"5 Mercer St Toronto, ON M5V 1H2 Canada","[[Monday, [['11:30 am--10:00 pm']]], [Tuesday,...",(416) 850-2855,False,"[43.645848, -79.389668]",Toronto
3,5.0,Stephen Carr,"Delicioso! All the food was great, from guacam...",[Mexican Restaurant],106946454276117525029,1.335926e+09,"May 1, 2012",103781603078729274814,Milagro,$,"5 Mercer St Toronto, ON M5V 1H2 Canada","[[Monday, [['11:30 am--10:00 pm']]], [Tuesday,...",(416) 850-2855,False,"[43.645848, -79.389668]",Toronto
4,5.0,Sev Dergo,This is our go to place for Mexican. Service i...,[Mexican Restaurant],106946454276117525029,1.385016e+09,"Nov 20, 2013",104190882158899538024,Milagro,$,"5 Mercer St Toronto, ON M5V 1H2 Canada","[[Monday, [['11:30 am--10:00 pm']]], [Tuesday,...",(416) 850-2855,False,"[43.645848, -79.389668]",Toronto
5,3.0,ramy rutu,Just good .,[Mexican Restaurant],106946454276117525029,1.352068e+09,"Nov 4, 2012",105136765670976622636,Milagro,$,"5 Mercer St Toronto, ON M5V 1H2 Canada","[[Monday, [['11:30 am--10:00 pm']]], [Tuesday,...",(416) 850-2855,False,"[43.645848, -79.389668]",Toronto
6,5.0,Joe Corson,I was on my way to one of the fine Italian res...,[Mexican Restaurant],106946454276117525029,1.354237e+09,"Nov 29, 2012",105525052822215095905,Milagro,$,"5 Mercer St Toronto, ON M5V 1H2 Canada","[[Monday, [['11:30 am--10:00 pm']]], [Tuesday,...",(416) 850-2855,False,"[43.645848, -79.389668]",Toronto
7,4.0,Andres Valera,One of the best mexican restaurants in the cit...,[Mexican Restaurant],106946454276117525029,1.377190e+09,"Aug 22, 2013",107661149558132736473,Milagro,$,"5 Mercer St Toronto, ON M5V 1H2 Canada","[[Monday, [['11:30 am--10:00 pm']]], [Tuesday,...",(416) 850-2855,False,"[43.645848, -79.389668]",Toronto
8,4.0,Matt Keoshkerian,Decent Mexican food this far north. A bit on t...,[Mexican Restaurant],106946454276117525029,1.318532e+09,"Oct 13, 2011",108012991939099820186,Milagro,$,"5 Mercer St Toronto, ON M5V 1H2 Canada","[[Monday, [['11:30 am--10:00 pm']]], [Tuesday,...",(416) 850-2855,False,"[43.645848, -79.389668]",Toronto
9,4.0,Afan Ahmed,None,[Mexican Restaurant],106946454276117525029,1.373664e+09,"Jul 12, 2013",109551735352504400822,Milagro,$,"5 Mercer St Toronto, ON M5V 1H2 Canada","[[Monday, [['11:30 am--10:00 pm']]], [Tuesday,...",(416) 850-2855,False,"[43.645848, -79.389668]",Toronto


In [82]:
data['categories'].isnull().sum()

3439

In [83]:
category = set()
for list in data['categories']:
    if list != None:
        category.update(list)

In [85]:
print("number of categories in only toronto:",len(category))

number of categories in only toronto: 1352


In [20]:
master_list = []
for i in range(places.shape):
    line = f.readline()
    line = line.replace("u'", '"')
    line = line.replace("'", '"')
    try:
        d = ast.literal_eval(line)
        buff = [d[key] for key in d.keys()]
        if buff[1].isascii() == True and buff[2].isascii() == True:
            master_list.append(buff)
    except Exception as e:
        pass

TypeError: object of type '_io.TextIOWrapper' has no len()

In [14]:
master_list = []
for i in range(100):
    line = f.readline()
    line = line.replace("u'", '"')
    line = line.replace("'", '"')
    try:
        d = ast.literal_eval(line)
        buff = [d[key] for key in d.keys()]
        if buff[1].isascii() == True and buff[2].isascii() == True:
            master_list.append(buff)
    except Exception as e:
        pass

<_io.TextIOWrapper name='../Datasets/places.json' mode='r' encoding='utf-8'>

In [41]:
f = open('file.json', encoding='utf-8')

In [42]:
master_list = []
for i in range(100):
    line = f.readline()
    line = line.replace("u'", '"')
    line = line.replace("'", '"')
    try:
        d = ast.literal_eval(line)
        buff = [d[key] for key in d.keys()]
        if buff[1].isascii() == True and buff[2].isascii() == True:
            master_list.append(buff)
    except Exception as e:
        pass

In [43]:
columns = [key for key in d.keys()]
master_df = pd.DataFrame(master_list, columns=columns)

39

True